<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/5-ImageClass/CatDog/modeltrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Datayı Githubdan hızlıca Colabe çek

In [ ]:
!wget https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/5-ImageClass/CatDog/data.zip -O data.zip

--2024-12-19 15:28:31--  https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/5-ImageClass/CatDog/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/onuralpArsln/MlAiTutorialProjects/main/5-ImageClass/CatDog/data.zip [following]
--2024-12-19 15:28:31--  https://raw.githubusercontent.com/onuralpArsln/MlAiTutorialProjects/main/5-ImageClass/CatDog/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46210370 (44M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  44.07M  --.-KB/s    in 0.1s    

2024-12-19 15:28:33 (298 MB/s) - ‘data.zip’ sa

Dosyayı zipten çıkart

In [ ]:
!unzip -q data.zip

In [ ]:
!pwd


/content


Burada, os modülünü ve TensorFlow'un image araçlarını kullanarak klasörleri tarayıp görüntüleri okuyoruz ve yeniden boyutlandırıyoruz. İşlem sırasında etiketler de belirleniyor (cats için 0, dogs için 1).

In [ ]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Verinin bulunduğu ana klasör
dataset_dir = 'test_set'

# Görselleri okuma ve etiketleme fonksiyonu
def load_images_from_directory(base_dir, label, img_size=(128, 128)):
    images = []
    labels = []
    for file_name in os.listdir(base_dir):
        file_path = os.path.join(base_dir, file_name)
        if file_name.endswith('.jpg'):
            # Görüntüyü oku ve yeniden boyutlandır
            img = tf.keras.preprocessing.image.load_img(file_path, target_size=img_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize et
            images.append(img_array)
            labels.append(label)
    return images, labels

# Görselleri yükle
cat_images, cat_labels = load_images_from_directory(os.path.join(dataset_dir, 'cats'), label=0)
dog_images, dog_labels = load_images_from_directory(os.path.join(dataset_dir, 'dogs'), label=1)

# Verileri birleştir
images = np.array(cat_images + dog_images)
labels = np.array(cat_labels + dog_labels)

# Veriyi train ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


Data Preprocessing - Resizing and Splitting
Bu aşamada, görüntüleri normalleştirdiğimizden emin olduk ve eğitim/test verilerini doğru şekilde böldük. Şimdi doğrulama verilerini eğitim verisinden ayırabiliriz:

In [ ]:
# Eğitim verilerinden bir kısmını doğrulama için ayır
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


Model Building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN modelini oluştur
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification için
])

# Modeli derle
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Train

In [ ]:
# Modeli eğit
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,  # Kendi ihtiyacınıza göre artırabilirsiniz
    batch_size=32
)
